## Score，and cross-validated scores
分数，和交叉验证分数

如我们所见，每个预测器能够提供一个score方法，用于判断新数据fit或者prediction的质量。Score的值越大越好。
看一个使用SVM判别digits数据集的范例：


In [1]:
from sklearn import datasets, svm
digits = datasets.load_digits()
X_digits = digits.data#观测值
y_digits = digits.target#预测值
svc = svm.SVC(C=1, kernel='linear')#创建SVC预测器
svc.fit(X_digits[:-100], y_digits[:-100]).score(X_digits[-100:], y_digits[-100:])


0.97999999999999998

为了更好的度量预测的竞速，我们可以将数据集分解成为几个部分，分别用于trainning和testing。

In [2]:
>>> import numpy as np
>>> X_folds = np.array_split(X_digits, 3)
>>> y_folds = np.array_split(y_digits, 3)
>>> scores = list()
>>> for k in range(3):
...     # We use 'list' to copy, in order to 'pop' later on
...     X_train = list(X_folds)
...     X_test  = X_train.pop(k)
...     X_train = np.concatenate(X_train)
...     y_train = list(y_folds)
...     y_test  = y_train.pop(k)
...     y_train = np.concatenate(y_train)
...     scores.append(svc.fit(X_train, y_train).score(X_test, y_test))
>>> print(scores)


[0.93489148580968284, 0.95659432387312182, 0.93989983305509184]


这被称为KFold 交叉验证方法。

## Cross-validation generators
交叉验证生成器

Scikit-learn有一系列类，可以用来为流行的交叉验证策略生成train/test索引列表。
这些类具有一个split方法，可以接受input数据集，然后将其分割，生成train/test集合索引，用于选定交叉验证策略的每一个迭代。
下面一个例子介绍了如何使用split方法。


In [3]:
>>> from sklearn.model_selection import KFold, cross_val_score
>>> X = ["a", "a", "b", "c", "c", "c"]
>>> k_fold = KFold(n_splits=3)
>>> for train_indices, test_indices in k_fold.split(X):
...      print('Train: %s | test: %s' % (train_indices, test_indices))


Train: [2 3 4 5] | test: [0 1]
Train: [0 1 4 5] | test: [2 3]
Train: [0 1 2 3] | test: [4 5]


然后交叉验证可以很容易的执行：

In [4]:
>>> [svc.fit(X_digits[train], y_digits[train]).score(X_digits[test], y_digits[test])
...          for train, test in k_fold.split(X_digits)]


[0.93489148580968284, 0.95659432387312182, 0.93989983305509184]

交叉验证分数能够直接使用cross_val_score助手计算得到。给定一个预测器，交叉验证对象和输入数据集，cross_val_score重复分割数据为一个training和testing集合。使用training集合训练预测器，然后基于testing集合为每个交叉迭代计算score。

默认地，预测器的score方法用于计算individual的score。
可以参考metrics 模块来学习更多可用的评判方法。


In [5]:
>>> cross_val_score(svc, X_digits, y_digits, cv=k_fold, n_jobs=-1)

array([ 0.93489149,  0.95659432,  0.93989983])

n_jobs = -1表示计算工作将在计算机的所有CPU上运行。
一个可选的参数scoring，可以用于指定其他的评判方法。


In [6]:
>>> cross_val_score(svc, X_digits, y_digits, cv=k_fold,
...                 scoring='precision_macro')


array([ 0.93969761,  0.95911415,  0.94041254])

下面列举一下常见的交叉验证生成器：
1、KFold (n_splits, shuffle, random_state)
分割为K的fold，在K-1个上训练，然后在剩余的数据上测试。
2、StratifiedKFold (n_splits, shuffle, random_state)
与KFold相同，但是在每个fold内保留类分布。
3、 GroupKFold (n_splits)
能够确保相同的group并同时在testing和training集合中
4、ShuffleSplit (n_splits, test_size, train_size, random_state)
根据随机排列生成train/test索引。
……


## Grid-search and cross-validated estimators

### Grid-search

scikit-learn提供了一个对象，给定数据，计算预测器在参数网格上拟合的分数，并选择参数以最大化交叉验证分数。这个对象在构建过程中需要一个预测器，并公开expose一个预测器API：


In [7]:
>>> from sklearn.model_selection import GridSearchCV, cross_val_score
>>> Cs = np.logspace(-6, -1, 10)
>>> clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs),
...                    n_jobs=-1)
>>> clf.fit(X_digits[:1000], y_digits[:1000])    


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-06,   3.59381e-06,   1.29155e-05,   4.64159e-05,
         1.66810e-04,   5.99484e-04,   2.15443e-03,   7.74264e-03,
         2.78256e-02,   1.00000e-01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [8]:
clf.best_score_

0.92500000000000004

In [9]:
clf.best_estimator_.C

0.0077426368268112772

In [11]:
>>> # Prediction performance on test set is not as good as on train set
>>> clf.score(X_digits[1000:], y_digits[1000:])      

0.94353826850690092

默认的，GridSearchCV使用一个3folds交叉验证。然而，如果它检测到传递来一个分类器，而不是回归器，那么就要使用分层3folds。（这里不太理解~）

### Nested cross-validation，嵌套交叉验证



In [12]:
cross_val_score(clf, X_digits, y_digits)

array([ 0.93853821,  0.96327212,  0.94463087])

并行两个交叉验证循环：一个是GridSearchCV预测器，设置gamma；另一个是通过cross_val_score，来衡量预测器的预测精度。结果分数是无偏估计得。

### Cross-validated estimators，交叉验证预测器
交叉验证设置参数可以在逐算法的基础上更高效地完成。这就是为何对于特定的预测器，scikit-learn自动的设置参数。



In [13]:
>>> from sklearn import linear_model, datasets
>>> lasso = linear_model.LassoCV()
>>> diabetes = datasets.load_diabetes()
>>> X_diabetes = diabetes.data
>>> y_diabetes = diabetes.target
>>> lasso.fit(X_diabetes, y_diabetes)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [14]:
>>> # The estimator chose automatically its lambda:
>>> lasso.alpha_ 


0.012291895087486161

这些具备交叉验证功能的预测器命名很简单，一般在后缀上有CV两个字母。